In [32]:
from DataLoading import ImagesLoader
from PreProcessing import PreProcessor
from FeatureExtraction import FeatureExtractor

from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import os
import numpy as np
import random
import cv2
import pickle
import matplotlib.pyplot as plt

# Data Loading

In [5]:
females_list = os.listdir('./data/Females')
females_list = [f"./data/Females/{img_path}" for img_path in females_list]
random.shuffle(females_list)

males_list = os.listdir('./data/Males')
males_list = [f"./data/Males/{img_path}" for img_path in males_list] 
random.shuffle(males_list)

imgs_list = females_list + males_list
y = np.array(([0] * len(females_list)) + ([1] * len(males_list)))

assert len(imgs_list) == len(females_list) + len(males_list)
assert len(y) == len(imgs_list)

In [31]:
print(f"Number of Female Images: {len(females_list)}")
print(f"Number of Male Images: {len(males_list)}")
print(f"Ratio of Females in The Dataset: {round(len(females_list) / len(imgs_list), 2) * 100}%")
print(f"Total Number of Images: {len(imgs_list)}")


Number of Female Images: 131
Number of Male Images: 232
Ratio of Females in The Dataset: 36.0%
Total Number of Images: 363


# Splitting Data

In [7]:
females_split = train_test_split(
    females_list, y[:len(females_list)], test_size=0.2,random_state=42)
 
males_spilt = train_test_split(
    males_list[:len(females_list)],y[len(females_list):len(females_list)*2], test_size=0.2,random_state=42)

X_train = females_split[0] + males_spilt[0] + males_list[len(females_list): len(females_list)+30]
X_test = females_split[1] + males_spilt[1]
y_train = np.concatenate([females_split[2], males_spilt[2], y[len(females_list)*2:(len(females_list)*2) + 30]])
y_test = np.concatenate([females_split[3], males_spilt[3]])
X_test_old = X_test

assert len(X_train) == len(y_train)
assert len(X_test) == len(y_test)

In [29]:
print(f"Number of validation examples: {len(X_test)}")

Number of validation examples: 54


# Images Loading

In [9]:
X_train_data = ImagesLoader(X_train)

In [10]:
X_test_data = ImagesLoader(X_test)

# Creating Model Pipeline

In [11]:
data_pipe = Pipeline([
    ('PreProcessing', PreProcessor()),
    ("Features", FeatureExtractor()),
])

In [12]:
model_pipe = Pipeline([
    ("Standard Scaling", StandardScaler()),
    ("PCA", PCA(n_components=80)),
    ("Estimator", SVC(C=3, class_weight={0:1.3, 1:1}))
])

# Training

In [13]:
X_train_transformed = data_pipe.transform(X_train_data)

In [24]:
data_pipe.fit(X_train_data, y_train)

Pipeline(steps=[('PreProcessing',
                 <PreProcessing.PreProcessor object at 0x0000029B7D0256A0>),
                ('Features',
                 <FeatureExtraction.FeatureExtractor object at 0x0000029B7D025400>)])

In [21]:
model_pipe.fit(X_train_transformed, y_train)

Pipeline(steps=[('Standard Scaling', StandardScaler()),
                ('PCA', PCA(n_components=80)),
                ('Estimator', SVC(C=3, class_weight={0: 1.3, 1: 1}))])

# Validation Results

In [15]:
from sklearn.metrics import classification_report

X_test_transformed = data_pipe.transform(X_test_data)

In [22]:
y_pred = model_pipe.predict(X_test_transformed)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.89      0.89        27
           1       0.89      0.89      0.89        27

    accuracy                           0.89        54
   macro avg       0.89      0.89      0.89        54
weighted avg       0.89      0.89      0.89        54



In [17]:
X_test_full = X_test + males_list[len(females_list)+30:]
y_test_full = np.concatenate([y_test, [1] * (len(X_test_full) - len(y_test))])

In [18]:
X_test_full_data = ImagesLoader(X_test_full)

In [19]:
X_test_transformed_full = data_pipe.transform(X_test_full_data)

In [23]:
y_pred = model_pipe.predict(X_test_transformed_full)
print(classification_report(y_test_full, y_pred))

              precision    recall  f1-score   support

           0       0.67      0.89      0.76        27
           1       0.97      0.88      0.92        98

    accuracy                           0.88       125
   macro avg       0.82      0.88      0.84       125
weighted avg       0.90      0.88      0.89       125

